### 1. Import required libraries

In [ ]:
import os
from flask import Flask, request, jsonify
from flask_cors import CORS
from dotenv import load_dotenv
from openai import OpenAI

##### What each import does:
- os → Access environment variables (used for API keys)
- Flask → Create a web server
- request → Read incoming HTTP request data
- jsonify → Send JSON responses
- CORS → Allow requests from other domains (e.g., frontend apps)
- load_dotenv → Load environment variables from a .env file
- OpenAI → OpenAI client to call GPT models

### 2. Load the OpenAI API key

In [ ]:
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

### 3. Create the OpenAI client

In [ ]:
client = OpenAI(api_key=OPENAI_API_KEY)

### 4. Initialize Flask app and enable CORS

In [ ]:
app = Flask(__name__)
CORS(app)

- Creates a Flask web application
- Enables CORS, allowing frontend apps (React, Vue, etc.) to call this API

### 5. Store conversation history in memory

In [ ]:
conversation_history = {}

- Python dictionary to store chat history
- Key = session_id
- Value = list of messages (system, user, assistant)

Example:

```json
{
  "abc123": [
    {"role": "system", "content": "..."},
    {"role": "user", "content": "Hello"},
    {"role": "assistant", "content": "Hi!"}
  ]
}
```

### 6. Create the /api/chat endpoint

In [ ]:
@app.route("/api/chat", methods=["POST"])
def chat():

    ## 7. Read incoming request data
    data = request.get_json()
    user_message = data.get("message")
    session_id = data.get("session_id", "default")

    ## 8. Validate user input
    if not user_message:
        return jsonify({"error": "Message is required"}), 400
    
    ## 9. Initialize a new conversation session
    if session_id not in conversation_history:
        conversation_history[session_id] = [
            {"role": "system", "content": "You are a helpful AI assistant."}
        ]
    
    ## 10. Store the user message
    conversation_history[session_id].append(
            {"role": "user", "content": user_message}
        )
    
    try:
        ## 11. Call OpenAI Chat Completion API
        response = client.chat.completions.create(
                    model="gpt-3.5-turbo",
                    messages=conversation_history[session_id],
                    temperature=0.7,
                )
        
        ## 12. Extract AI response
        reply = response.choices[0].message.content.strip()

        ## 13. Save assistant response
        conversation_history[session_id].append(
                {"role": "assistant", "content": reply}
            )


        ## 14. Return response to client
        return jsonify({"reply": reply})
    
    ## 15. Handle errors safely
    except Exception as e:
        return jsonify({"error": str(e)}), 500


### 16. Create /api/reset endpoint

In [ ]:
@app.route("/api/reset", methods=["POST"])
def reset():
    data = request.get_json()
    session_id = data.get("session_id", "default")
    ## 17. Reset conversation history
    if session_id in conversation_history:
        del conversation_history[session_id]
    return jsonify({"status": "reset"})

### 18. Start the Flask server (Development)

In [ ]:
if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5000, debug=True)

### Run Flask in background thread (IMPORTANT)

In [9]:
import threading
def run_flask():
    app.run(
        host="127.0.0.1",
        port=5000,
        debug=False,
        use_reloader=False
    )

thread = threading.Thread(target=run_flask)
thread.daemon = True
thread.start()

print("Flask server started")


Flask server started


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


### Test the API

In [10]:
import requests

response = requests.post(
    "http://127.0.0.1:5000/api/chat",
    json={
        "message": "Hello, explain what you are",
        "session_id": "test1"
    },
    timeout=30
)

response.json()


127.0.0.1 - - [09/Jan/2026 10:44:08] "POST /api/chat HTTP/1.1" 200 -


{'reply': "Hello! I am an AI assistant here to help answer your questions and provide information on a variety of topics. Feel free to ask me anything you'd like to know!"}

### Test memory (multi-turn)

In [11]:
requests.post(
    "http://127.0.0.1:5000/api/chat",
    json={
        "message": "What did I just ask?",
        "session_id": "test1"
    }
).json()


127.0.0.1 - - [09/Jan/2026 10:46:46] "POST /api/chat HTTP/1.1" 200 -


{'reply': 'You asked me to explain what I am, referring to my role as an AI assistant.'}

In [12]:
requests.post(
    "http://127.0.0.1:5000/api/chat",
    json={
        "message": "Who is the prime minister of Bangladesh?",
        "session_id": "test1"
    }
).json()

127.0.0.1 - - [09/Jan/2026 10:50:04] "POST /api/chat HTTP/1.1" 200 -


{'reply': 'As of my last update, the Prime Minister of Bangladesh is Sheikh Hasina. She has been in office since January 2009.'}